In [13]:
import pandas as pd
import numpy as np
import re

In [14]:
df = pd.read_csv("zdruzen_df_brez_dolzine-2.csv", index_col="SR", sep=",", )
df.head()

,NAZIV_SR,DATUM_NASTANKA_SR,STATUS,WORKLOGID,OPIS,DOLGI_OPIS_X
SR,,,,,,
SR271424,3-344158,2024-01-03 06:40:03.759000,CLOSED,466836.0,Rešitev,zahteva na oddano LP Goran !
SR271425,3-137428,2024-01-03 07:00:02.971000,CLOSED,466837.0,Rešitev,stornirajte spremembo in pričnite ponovno s pr...
SR271425,3-137428,2024-01-03 07:00:02.971000,CLOSED,466899.0,Potrjujem rešitev,NaN
SR271426,Kadri - prva posodobitev ob vstopu,2024-01-03 07:25:14.172000,CLOSED,466840.0,Sporočilo Informatika - 3.1.2024,Zdravo - pogledala sem nastavitve ure v PROD i...
SR271426,Kadri - prva posodobitev ob vstopu,2024-01-03 07:25:14.172000,CLOSED,467059.0,Rešitev,Časovne nastavitve smo popravili. Danes ni bil...


In [15]:
generic_phrases = [
    "Potrjujem rešitev",
    "Urejeno",
    "Urejeno.",
    "Urejeno !",
    "Pozdravljeni. Urejeno.",
    "Pozdravljeni. Urejeno. Lep pozdrav, Niko !",
    "Pozdravljeni",
    "popravljeno LP Goran !",
    "Popravljeno LP Goran !",
    "urejeno LP Goran !",
    "Urejeno. Lp, Mihaela !",
    "Urejeno. !",
    "urejeno po telefonu !",
    "Pozdravljeni! Zahtevek smo sprejli v reševenje. Lep pozdrav! Nataša !",
    "Urejeno, !",
    "Urejeno.!",
    "Urejeno. !",
    "urejeno !",
    "Zdravo",
    "LP Goran !",
    "Pozdravljeni, vaš zahtevek je v obravnavi. Lp Marko !",
    ""
]

In [16]:
escaped = [re.escape(p.strip()) for p in generic_phrases if p.strip()]
pattern_exact = r'^(?:' + '|'.join(escaped) + r')[\.\!\,\s]*$'       # za ujemanje »celotni niz je fraza«
pattern_inside = r'\b(?:' + '|'.join(escaped) + r')[\.\!\,\s]*'      # za izrezovanje fraze znotraj besedila

In [17]:
df["DOLGI_OPIS_X"] = (
    df["DOLGI_OPIS_X"]
      .astype(str)                   # za varnost, če so kje NaN
      .apply(lambda x: re.sub(pattern_inside, '', x, flags=re.IGNORECASE))
      .str.strip()                   # odstrani odvečne presledke na začetku/koncu
)


In [18]:
rows_to_drop = df["DOLGI_OPIS_X"].str.match(pattern_exact, case=False, na=True) | df["DOLGI_OPIS_X"].isna()
df_clean = df.loc[~rows_to_drop].copy()

In [19]:
df_clean.to_csv("opis_cleaned.csv", index=True)

In [20]:
print(f"Izvirno število vrstic: {len(df):,}")
print(f"Po čiščenju:            {len(df_clean):,}")
print(df_clean["DOLGI_OPIS_X"].head())
df_clean.head(10)

Izvirno število vrstic: 28,353
Po čiščenju:            28,353
SR
SR271424                                    zahteva na oddano
SR271425    stornirajte spremembo in pričnite ponovno s pr...
SR271425                                                  nan
SR271426    - pogledala sem nastavitve ure v PROD in UAT B...
SR271426    Časovne nastavitve smo popravili. Danes ni bil...
Name: DOLGI_OPIS_X, dtype: object


,NAZIV_SR,DATUM_NASTANKA_SR,STATUS,WORKLOGID,OPIS,DOLGI_OPIS_X
SR,,,,,,
SR271424,3-344158,2024-01-03 06:40:03.759000,CLOSED,466836.0,Rešitev,zahteva na oddano
SR271425,3-137428,2024-01-03 07:00:02.971000,CLOSED,466837.0,Rešitev,stornirajte spremembo in pričnite ponovno s pr...
SR271425,3-137428,2024-01-03 07:00:02.971000,CLOSED,466899.0,Potrjujem rešitev,nan
SR271426,Kadri - prva posodobitev ob vstopu,2024-01-03 07:25:14.172000,CLOSED,466840.0,Sporočilo Informatika - 3.1.2024,- pogledala sem nastavitve ure v PROD in UAT B...
SR271426,Kadri - prva posodobitev ob vstopu,2024-01-03 07:25:14.172000,CLOSED,467059.0,Rešitev,Časovne nastavitve smo popravili. Danes ni bil...
SR271426,Kadri - prva posodobitev ob vstopu,2024-01-03 07:25:14.172000,CLOSED,467060.0,Potrjujem rešitev,nan
SR271427,"Enotna vloga, Projektni pogoji, SkP, SzP, PoP",2024-01-03 07:29:10.194000,CLOSED,467414.0,Rešitev,lahko se navežete na postopek. Samo še v info:...
SR271428,Izostala arhiviranja preteklega tedna,2024-01-03 07:30:02.831000,CLOSED,466887.0,Rešitev,
SR271429,Ureditev naloga odklopa na 2-158443,2024-01-03 07:30:07.940000,CLOSED,466931.0,NaN,nisem prepričan kaj je narobe? Kakšno napako j...


In [26]:
from pathlib import Path
import re
import textwrap
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [27]:
_THIS_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
DATA_PATH = _THIS_DIR / "opis_cleaned.csv"
MODEL_PATH = _THIS_DIR / "best_sr_status_model.pkl"

def normalize(text: str) -> str:
    """Basic normalisation/clean‑up for Slovenian text."""
    if not isinstance(text, str):
        return ''
    txt = text.lower()
    # remove digits and punctuation
    txt = re.sub(r'\d+', ' ', txt)
    txt = re.sub(r'[\W_]+', ' ', txt, flags=re.UNICODE)
    txt = re.sub(r'\s{2,}', ' ', txt)
    return txt.strip()

def load_data() -> pd.DataFrame:
    df = pd.read_csv(DATA_PATH)
    # keep only rows that have STATUS and long description
    df = df.dropna(subset=['STATUS', 'DOLGI_OPIS_X', 'NAZIV_SR'])
    # basic normalisation
    df['opis_norm'] = df['DOLGI_OPIS_X'].apply(normalize)
    # filter out empty descriptions
    df = df[df['opis_norm'].str.len() > 0]
    # keep statuses with at least 100 samples
    counts = df['STATUS'].value_counts()
    frequent = counts[counts >= 100].index
    df = df[df['STATUS'].isin(frequent)].copy()
    return df

def evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'LogReg': LogisticRegression(max_iter=1000, class_weight='balanced'),
        'LinearSVC': LinearSVC(class_weight='balanced'),
        'MultinomialNB': MultinomialNB()
    }
    results = {}
    for name, clf in models.items():
        pipe = Pipeline([
            ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_features=50_000)),
            ('clf', clf)
        ])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        results[name] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_macro': f1_score(y_test, y_pred, average='macro'),
            'report': classification_report(y_test, y_pred, zero_division=0)
        }
    return results

def select_and_train_best(df):
    X_train, X_test, y_train, y_test = train_test_split(
        df['opis_norm'], df['STATUS'], test_size=0.2, stratify=df['STATUS'], random_state=42
    )
    results = evaluate_models(X_train, X_test, y_train, y_test)
    # choose best by macro‑F1
    best_name = max(results, key=lambda k: results[k]['f1_macro'])
    best_clf = {
        'LogReg': LogisticRegression(max_iter=1000, class_weight='balanced'),
        'LinearSVC': LinearSVC(class_weight='balanced'),
        'MultinomialNB': MultinomialNB()
    }[best_name]
    best_pipe = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_features=50_000)),
        ('clf', best_clf)
    ])
    best_pipe.fit(df['opis_norm'], df['STATUS'])
    joblib.dump({'model': best_pipe, 'meta': {'best_algo': best_name, 'results': results}}, MODEL_PATH)
    return results, best_name

def predict_status(naziv_sr: str):
    """Return tuple (predicted_STATUS, long_description) for given NAZIV_SR."""
    bundle = joblib.load(MODEL_PATH)
    model = bundle['model']
    df = load_data()
    row = df[df['NAZIV_SR'] == naziv_sr]
    if row.empty:
        raise ValueError(f'No entry with NAZIV_SR = {naziv_sr!r}')
    opis = row.iloc[0]['opis_norm']
    status_pred = model.predict([opis])[0]
    return status_pred, row.iloc[0]['DOLGI_OPIS_X']

def main():
    df = load_data()
    results, best_name = select_and_train_best(df)
    print('\nEvaluation results (test set):')
    for algo, res in results.items():
        print(f"\n=== {algo} ===")
        print(f"Accuracy: {res['accuracy']:.3f}")
        print(f"F1‑macro: {res['f1_macro']:.3f}")
    print(f"\nBest algorithm: {best_name}. Model saved to {MODEL_PATH}")
    # demo
    demo_key = df['NAZIV_SR'].iloc[0]
    pred, desc = predict_status(demo_key)
    print(f"\nDEMO --> NAZIV_SR: {demo_key}\nPredicted STATUS: {pred}\nDOLGI_OPIS_X: {textwrap.shorten(desc, 120)}")

if __name__ == '__main__':
    main()


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



Evaluation results (test set):

=== LogReg ===
Accuracy: 0.784
F1‑macro: 0.334

=== LinearSVC ===
Accuracy: 0.962
F1‑macro: 0.382

=== MultinomialNB ===
Accuracy: 0.966
F1‑macro: 0.246

Best algorithm: LinearSVC. Model saved to /content/best_sr_status_model.pkl

DEMO --> NAZIV_SR: 3-344158
Predicted STATUS: CLOSED
DOLGI_OPIS_X: zahteva na oddano


In [31]:
from __future__ import annotations

from pathlib import Path
import re
import textwrap
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score


In [32]:
_THIS_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
DATA_PATH = _THIS_DIR / "opis_cleaned.csv"
MODEL_PATH = _THIS_DIR / "best_sr_status_model.pkl"

CANDIDATE_MODELS = {
    "LogReg": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "LinearSVC": LinearSVC(class_weight="balanced"),
    "MultinomialNB": MultinomialNB(),
}

In [33]:

def normalize(text: str) -> str:
    """Very light normalisation (lower‑case, remove digits & punctuation)."""
    if not isinstance(text, str):
        return ""
    txt = text.lower()
    txt = re.sub(r"\d+", " ", txt)
    txt = re.sub(r"[\W_]+", " ", txt, flags=re.UNICODE)
    txt = re.sub(r"\s{2,}", " ", txt)
    return txt.strip()

In [34]:

def load_data() -> pd.DataFrame:
    if not DATA_PATH.exists():
        raise FileNotFoundError(f"CSV not found at {DATA_PATH}")

    df = pd.read_csv(DATA_PATH)
    required = {"DOLGI_OPIS_X", "NAZIV_SR", "STATUS"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV is missing columns: {', '.join(sorted(missing))}")

    df.dropna(subset=list(required), inplace=True)
    df["opis_norm"] = df["DOLGI_OPIS_X"].apply(normalize)
    df = df[df["opis_norm"].str.len() > 0]

    # (primer: filtriraj redke statuse, da ne rušijo klasiifikacije)
    counts = df["STATUS"].value_counts()
    frequent = counts[counts >= 100].index
    if len(frequent) < len(counts):
        df = df[df["STATUS"].isin(frequent)].copy()

    return df

In [ ]:
def _evaluate(X_train, X_test, y_train, y_test):
    """Train every candidate model & gather accuracy + macro‑F1."""
    results: dict[str, dict] = {}
    for name, clf in CANDIDATE_MODELS.items():
        pipe = Pipeline([
            ("tfidf", TfidfVectorizer(ngram_range=(1, 3), min_df=3, max_features=50_000)),
            ("clf", clf),
        ])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        results[name] = {
            "accuracy": accuracy_score(y_test, y_pred),
            "f1_macro": f1_score(y_test, y_pred, average="macro"),
            "report": classification_report(y_test, y_pred, zero_division=0),
            "pipeline": pipe,
        }
    return results

In [35]:
def train(force: bool = False):
    """Train (or load) model. Returns meta‑info & best algo name."""
    if MODEL_PATH.exists() and not force:
        bundle = joblib.load(MODEL_PATH)
        return bundle["meta"], bundle["meta"]["best_algo"]

    df = load_data()
    X_train, X_test, y_train, y_test = train_test_split(
        df["opis_norm"], df["STATUS"], test_size=0.2, stratify=df["STATUS"], random_state=42
    )

    results = _evaluate(X_train, X_test, y_train, y_test)
    best_name = max(results, key=lambda k: results[k]["f1_macro"])
    best_pipe = results[best_name]["pipeline"]

    # shrink results to only scalars for JSON serialisation
    meta = {k: {"accuracy": v["accuracy"], "f1_macro": v["f1_macro"]} for k, v in results.items()}
    meta["best_algo"] = best_name

    joblib.dump({"model": best_pipe, "meta": meta}, MODEL_PATH)
    return meta, best_name

In [36]:

def _load_bundle():
    if not MODEL_PATH.exists():
        raise RuntimeError("Model not found. Call train() first.")
    return joblib.load(MODEL_PATH)

In [37]:
def predict_status(naziv_sr: str):
    """Return `(predicted_STATUS, original_long_description)` for a given NAZIV_SR."""
    bundle = _load_bundle()
    model = bundle["model"]
    df = load_data()

    row = df.loc[df["NAZIV_SR"] == naziv_sr]
    if row.empty:
        raise ValueError(f"No entry with NAZIV_SR = {naziv_sr!r}")

    opis_norm = row.iloc[0]["opis_norm"]
    status_pred = model.predict([opis_norm])[0]

    return status_pred, row.iloc[0]["DOLGI_OPIS_X"]

In [45]:
def _cli_demo():
    # 1) (re)train model – force=True zagotovi svežo meta-strukturo
    meta, best_name = train(force=True)

    print("\n*** Evaluation (macro-F1) ***")
    for algo, scores in meta.items():
        if algo == "best_algo":
            continue
        acc = scores.get("accuracy", float("nan"))
        f1  = scores.get("f1_macro", float("nan"))
        print(f"{algo:<12}: acc={acc:.3f}  f1={f1:.3f}")
    print(f">> Best algorithm = {best_name}  (saved to {MODEL_PATH.name})")

    # 2) naključno izberi en primer iz TEST množice
    df = load_data()
    _, X_test, _, _ = train_test_split(
        df["opis_norm"], df["STATUS"],
        test_size=0.20,
        stratify=df["STATUS"],
        random_state=42,
    )
    test_df = df.loc[X_test.index]
    rnd_row = (
        test_df.sample(
            1,
            random_state=np.random.randint(1, 1_000_000),
        ).iloc[0]
    )

    sr_key      = rnd_row["NAZIV_SR"]
    true_status = rnd_row["STATUS"]
    pred_status, _ = predict_status(sr_key)

    print("\n--- Random sample from TEST split ---")
    print(f"NAZIV_SR        : {sr_key}")
    print(f"True STATUS     : {true_status}")
    print(f"Predicted STATUS: {pred_status}")
    print("Opis (120 zn.)  :", textwrap.shorten(rnd_row["DOLGI_OPIS_X"], 120))

    # 3) interaktivna zanka: ročni vnos NAZIV_SR
    print("\nTry it yourself – enter a NAZIV_SR (empty = quit).")
    while True:
        sr = input("> NAZIV_SR: ").strip()
        if not sr:
            break
        try:
            pred, desc = predict_status(sr)
            print(f"  Predicted STATUS: {pred}\n"
                  f"  Opis: {textwrap.shorten(desc, 120)}\n")
        except ValueError as e:
            print(f"  {e}\n")

_cli_demo()




*** Evaluation (macro-F1) ***
LogReg      : acc=0.784  f1=0.334
LinearSVC   : acc=0.962  f1=0.382
MultinomialNB: acc=0.966  f1=0.246
>> Best algorithm = LinearSVC  (saved to best_sr_status_model.pkl)

--- Random sample from TEST split ---
NAZIV_SR        : MX - MR ZAhtevki
True STATUS     : CLOSED
Predicted STATUS: CLOSED
Opis (120 zn.)  : Kot razloženo in prikazano preko Teamsov, je bila težava v nezagonu integracijske točke D365-MX Delovni nalogi [...]

Try it yourself – enter a NAZIV_SR (empty = quit).
> NAZIV_SR: IT - eIS
  Predicted STATUS: CLOSED
  Opis: podatki na soglasju so popravljeni, nadaljujte postopek

> NAZIV_SR: FW: Napaka v MX
  Predicted STATUS: CLOSED
  Opis: rezervacija odstranjena. !



KeyboardInterrupt: Interrupted by user

In [ ]:
_cli_demo()